### Real, Complex and Symplectic Reflection Groups - March 2023, RUB

## Computational Aspects of Complex Reflection Groups

Götz Pfeiffer - University of Galway

# 2. Orbits: Finite Coxeter Groups in Action

[![Vitruvian Man](images/vitruvian.jpg)](https://en.wikipedia.org/wiki/Vitruvian_Man)

## Setup

We use the graph traversal techniques from the last day to
* construct a finite Coxeter group as a permutation group,
* compute basic properties of elements and parabolic subgroups,
* determine the normalizer complement of a parabolic subgroup,
* examine the conjugation action with respect to element length.

First, reload the algorithms ...

In [3]:
Read("orbits.g");
LoadPackage("jupyterviz");
opts := rec(vertexwidth := 12, vertexheight := 12, edgecolor := "#def");;

true

## Finite Coxeter Groups

<div class="alert alert-danger">

**Definition.**

* Let $S$ be a finite set, and let $M = (m_{st})$ be a symmetric matrix with $m_{ss} = 1$
and $m_{st} = m_{ts} \in \{2,3,4,\dots,\infty\}$ if $s \neq t$.
* A **Coxeter group** with  **Coxeter Matrix** $M$ is the group $W$ given by the presentation
    $$
    W = \langle S \mid (st)^{m_{st}} = 1;\, s,t \in S \rangle
    $$
</div>

* The presentation of a Coxeter group can be described by a graph with vertex set $S$ and edges labelled $m_{st}$  between $s$ and $t$ if $m_{st} > 2$, where for $m_{st} = 3$ the label is usually omitted.
* A Coxeter group is called **simply laced** if $m_{st} \leq 3$ for all $s, t \in S$.
* Coxeter graphs (for simply laced irreducible finite Coxeter groups) can be made by the following GAP program.

In [4]:
coxeterGraph := function(series, rank)
    local edges;
    edges := List([2..rank], j -> [j-1, j]);
    if series >= "D" then  edges[1][2] := 3;  fi;
    if series >= "E" then  edges[2][2] := 4;  fi;
    return edges;
end;

function( series, rank ) ... end

* For example:

In [5]:
graph := coxeterGraph("A", 7);

[ [ 1, 2 ], [ 2, 3 ], [ 3, 4 ], [ 4, 5 ], [ 5, 6 ], [ 6, 7 ] ]

In [6]:
PlotGraph(graph, opts);

* A finite Coxeter group is a (real) **reflection group**, i.e., a group generated by reflections, as follows.
* From the Coxeter matrix $M$, or the corresponding graph, we set up a **Cartan matrix** $C$:
* $C = (c_{st})$ with $c_{ss} = 2$ and $c_{st} c_{ts} = 4 \cos^2 \frac{\pi}{m_{st}}$, $c_{st} = 0 \iff c_{ts} = 0$, $s \neq t$.

In [7]:
cartanMat := function(series, rank)
    local cartan,  ij,  i,  j;
    cartan := 2*IdentityMat(rank);;
    for ij in coxeterGraph(series, rank) do
        i := ij[1];  j := ij[2];
        cartan[i][j] := -1;
        cartan[j][i] := -1;
    od;
    if series = "B" then  cartan[1][2] := -2;  fi;
    if series = "C" then  cartan[2][1] := -2;  fi;
    if series = "F" then  cartan[3][4] := -2;  fi;  # sic!
    return cartan;
end;

function( series, rank ) ... end

In [9]:
C := cartanMat("A", 3);;
PrintArray(C);

[ [   2,  -1,   0 ],
  [  -1,   2,  -1 ],
  [   0,  -1,   2 ] ]


* From the Cartan matrix $C$ we can compute matrices for the **simple reflections** $s \in S$: the space $V = \mathbb{R}^n$ has a basis of **simple roots** $\{\alpha_s : s \in S\}$ and
$$
\alpha_t.\tilde{s} = \alpha_t - c_{st} \alpha_s
$$
defines a linear action $\tilde{s} \in \mathrm{GL}(V)$ of $s$ as **reflection** in the hyperplane orthogonal to $\alpha_s$.

In [13]:
S := [1..Length(C)];;  mats := [];;  one := C^0;;
for s in S do
    mats[s] := C^0;  mats[s]{S}[s] := one[s] - C[s];
od;

In [14]:
for mat in mats do
    PrintArray(mat);  Print("\n");
od;

[ [  -1,   0,   0 ],
  [   1,   1,   0 ],
  [   0,   0,   1 ] ]

[ [   1,   1,   0 ],
  [   0,  -1,   0 ],
  [   0,   1,   1 ] ]

[ [   1,   0,   0 ],
  [   0,   1,   1 ],
  [   0,   0,  -1 ] ]



In [15]:
mats[1]^2;

[ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ] ]

* Thus, $W = \langle \tilde{s} : s \in S \rangle \leq \mathrm{GL}(V)$ acts as a **reflection group** on $V$.
* Note that $\{\alpha_s : s \in S\}$ is not the standard basis of $V$.
* But there are real numbers $d_s$, $s \in S$ such that
$$
(\alpha_s, \alpha_t) = \tfrac12 d_s^2 c_{st}
$$
defines a $W$-invariant bilinear form on $V$ (with $d_s = \sqrt{(\alpha_s, \alpha_s)}$).

###  Root System

* The **root system** $\Phi$ of $W$ (or of $C$) is $\Phi = \{\alpha_s.w : s \in S,\,w \in W\}$, a union of $W$-orbits.
* So we can grow this root system as **orbits** of the simple roots $\alpha_s$ (which, as basis vectors of $\mathbb{R}^n$,  are simply the rows of the identity matrix $C^0$).

In [16]:
phi := orbits(mats, C^0, OnRight);

[ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ], [ -1, 0, 0 ], [ 1, 1, 0 ],   [ 0, -1, 0 ], [ 0, 1, 1 ], [ 0, 0, -1 ], [ -1, -1, 0 ], [ 1, 1, 1 ],   [ 0, -1, -1 ], [ -1, -1, -1 ] ]

* Note how $\Phi = \Phi^{+} \cup \Phi^{-}$, where $\Phi^{+} = \{ \alpha \in \Phi : \alpha \geq 0\}$.
* Or, how roots come in pairs of negatives $\{r, -r\}$.
* Thus it suffices to only enumerate the positive representatives of each pair ...

In [19]:
absRoot := root -> SignInt(Sum(root)) * root;;
onRoots := function(x, a)
    return absRoot(OnRight(x, a));
end;;
phi := orbits(mats, C^0, onRoots);

[ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 1, 0 ], [ 0, 1, 1 ],   [ 1, 1, 1 ] ]

* ... and find $\Phi$ as concatentation of $\Phi^{+}$ and $- \Phi^{+}$.

In [20]:
phi := Concatenation(phi, -phi);

[ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 1, 0 ], [ 0, 1, 1 ],   [ 1, 1, 1 ], [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ], [ -1, -1, 0 ],   [ 0, -1, -1 ], [ -1, -1, -1 ] ]

* In fact, for future reference, we can compute the edges of the action graph, and words in the generators along with $\Phi$. (What are suitable words for the initial roots? Let's take `[i]` for $\alpha_i$.  What are the initial edges?  There are none.)

In [21]:
orbits_with_words_and_edges := function(aaa, xxx, under)
    local   list,  edges,  words,  i,  k,  l,  z;
    words := List([1..Length(xxx)], i -> [i]);
    list := ShallowCopy(xxx);  edges := [];  i := 0;
    while i < Length(list) do
        i := i+1;
        for k in [1..Length(aaa)] do
            z := under(list[i], aaa[k]);
            l := Position(list, z);
            if l = fail then
                Add(list, z);
                Add(words, onWords(words[i], k));
                l := Length(list);
            fi;
            Add(edges, [i, l]);
        od;
    od;
    return rec(list := list, edges := edges, words := words);
end;

function( aaa, xxx, under ) ... end

In [23]:
roots := orbits_with_words_and_edges(mats, C^0, onRoots);;
roots.list;

[ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 1, 0 ], [ 0, 1, 1 ],   [ 1, 1, 1 ] ]

In [25]:
edges := Filtered(roots.edges, x-> x[1] <> x[2]);
PlotGraph(edges, opts);

[ [ 1, 4 ], [ 2, 4 ], [ 2, 5 ], [ 3, 5 ], [ 4, 2 ], [ 4, 1 ], [ 4, 6 ],   [ 5, 6 ], [ 5, 3 ], [ 5, 2 ], [ 6, 5 ], [ 6, 4 ] ]

In [26]:
roots.words;

[ [ 1 ], [ 2 ], [ 3 ], [ 1, 2 ], [ 2, 3 ], [ 1, 2, 3 ] ]

### Permutations

* Express simple reflections as permutations of the roots. Formula?  (Compare with Schreier generators $(f_y a)/f_{y.a}$!)
* The GAP function `Sortex` returns the permutation needed to sort a list: `Permuted(list, Sortex(list))` is the sorted list.
* So, if $a$ maps `xxx` to `yyy`, $b$ sorts `yyy` (i.e., maps it to `zzz`) and $c$ sorts `xxx` (i.e., maps it to `zzz`) then $c/b$ maps `xxx` to `yyy`.
* The permutation of `xxx` caused is the map that assigns $i \mapsto j$ if `yyy[i] = xxx[j]`, i.e. $b/c = (c/b)^{-1}$ (Check!).

In [27]:
permutation := function(a, xxx, under)
    return Sortex(List(xxx, x-> under(x, a))) / Sortex(ShallowCopy(xxx));
end;

function( a, xxx, under ) ... end

In [28]:
permutation(mats[1], phi, OnRight);

(1,7)(2,4)(5,6)(8,10)(11,12)

* GAP has a function `Permutation` for this purpose.

In [29]:
Permutation(mats[1], phi, OnRight);

(1,7)(2,4)(5,6)(8,10)(11,12)

In [30]:
perms := List(mats, m-> Permutation(m, phi, OnRight));

[ (1,7)(2,4)(5,6)(8,10)(11,12), (1,4)(2,8)(3,5)(7,10)(9,11),   (2,5)(3,9)(4,6)(8,11)(10,12) ]

###  Coxeter Group

* Let's wrap all of the above into a GAP function that constructs a Coxeter group from its Cartan matrix.
* It will be convenient to register an **attribute** `Data` with GAP's permutation groups, as a place to store some Coxeter group data like the root system `phi` and the matrices `mats`. 

In [32]:
DeclareAttribute("Data", IsPermGroup);

coxeterGroup := function(C)
    local  one,  mats,  roots,  S,  s,  phi,  data,  G;
    one := C^0;  mats := [];  S := [1..Length(C)];
    for s in S do
        mats[s] := C^0;  mats[s]{S}[s] := one[s] - C[s];
    od;
    roots := orbits_with_words_and_edges(mats, C^0, onRoots);
    data := rec(mats := mats, roots := roots, rank := Length(S));
    data.N := Length(roots.list);
    data.phi := Concatenation(roots.list, -roots.list);
    data.perms := List(mats, m -> Permutation(m, data.phi, OnRight));
    G := GroupWithGenerators(data.perms);
    SetData(G, data);
    return G;
end;

function( C ) ... end

In [33]:
W:= coxeterGroup(C);

Group([ (1,7)(2,4)(5,6)(8,10)(11,12), (1,4)(2,8)(3,5)(7,10)(9,11), (2,5)(3,9)  (4,6)(8,11)(10,12) ])

In [34]:
Data(W);

rec( N := 6, mats := [ [ [ -1, 0, 0 ], [ 1, 1, 0 ], [ 0, 0, 1 ] ], [ [ 1, 1, 0 ], [ 0, -1, 0 ], [ 0, 1, 1 ] ], [ [ 1, 0, 0 ], [ 0, 1, 1 ], [ 0, 0, -1 ] ] ], perms := [ (1,7)(2,4)(5,6)(8,10)(11,12), (1,4)(2,8)(3,5)(7,10)(9,11), (2,5)(3,9)(4,6)(8,11)(10,12) ], phi := [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 1, 0 ], [ 0, 1, 1 ], [ 1, 1, 1 ], [ -1, 0, 0 ], [ 0, -1, 0 ], [ 0, 0, -1 ], [ -1, -1, 0 ], [ 0, -1, -1 ], [ -1, -1, -1 ] ], rank := 3, roots := rec( edges := [ [ 1, 1 ], [ 1, 4 ], [ 1, 1 ], [ 2, 4 ], [ 2, 2 ], [ 2, 5 ], [ 3, 3 ], [ 3, 5 ], [ 3, 3 ], [ 4, 2 ], [ 4, 1 ], [ 4, 6 ], [ 5, 6 ], [ 5, 3 ], [ 5, 2 ], [ 6, 5 ], [ 6, 6 ], [ 6, 4 ] ], list := [ [ 1, 0, 0 ], [ 0, 1, 0 ], [ 0, 0, 1 ], [ 1, 1, 0 ], [ 0, 1, 1 ], [ 1, 1, 1 ] ], words := [ [ 1 ], [ 2 ], [ 3 ], [ 1, 2 ], [ 2, 3 ], [ 1, 2, 3 ] ] ) )

In [35]:
sizeOfGroup(W);

24

* $E_8$, for example ...

In [37]:
E8:= coxeterGroup(cartanMat("E", 8));;
sizeOfGroup(E8);

696729600

* ... and its positive roots

In [39]:
edges := Filtered(Data(E8).roots.edges, x-> x[1] <> x[2]);;
PlotGraph(edges);

### Reflections

* From the word $w = a_0 a_1 \dots a_m$ corresponding to the root $\alpha \in \Phi^+$, we can compute the reflection
$$
s_{\alpha} = s_{a_0}^{s_{a_{1}} \dotsm s_{a_{m}}}
$$

In [40]:
reflections := function(W)
    local reflection_from_word;
    reflection_from_word := function(w)
        return W.(w[1])^Product(Data(W).perms{w{[2..Length(w)]}});
    end;
    return List(Data(W).roots.words, reflection_from_word);
end;

function( W ) ... end

* Example:

In [42]:
A3 := coxeterGroup(cartanMat("A", 3));;
rr := reflections(A3);

[ (1,7)(2,4)(5,6)(8,10)(11,12), (1,4)(2,8)(3,5)(7,10)(9,11),   (2,5)(3,9)(4,6)(8,11)(10,12), (1,8)(2,7)(3,6)(4,10)(9,12),   (1,6)(2,9)(3,8)(5,11)(7,12), (1,11)(3,10)(4,9)(5,7)(6,12) ]

In [43]:
Length(rr);

6

##  Basic Properties

* Some Basis properties of elements of $W$ are now immediate.

### Length

* The **length** of  an element $w \in W$ is $\ell(w) = \#\{\alpha \in \Phi^{+} : \alpha.w \in \Phi^{-}\}$

In [44]:
coxeterLength := function(W, w)
    return Number([1..Data(W).N], i-> i^w > Data(W).N);
end;

function( W, w ) ... end

In [45]:
List(Data(E8).perms, w-> coxeterLength(E8, w));

[ 1, 1, 1, 1, 1, 1, 1, 1 ]

### Products

* Conversion `word -> perm`: as a product of simple reflections

In [48]:
word := [5,2,3,8,7,5,3,1,6];;
perm := Product(Data(E8).perms{word});
coxeterLength(E8, perm);

(1,121)(2,122)(3,9)(4,10)(5,13,21,29)(6,7,8,142)(11,23)(12,26,28,42)(14,15,134,135)(16,17)(18,20,34,36)(19,38,35,54)(22,126,127,128)(24,33,39,50)(25,31,41,47)(27,46,43,30)(32,45,49,60)(37,40,53,56)(44,51,58,65)(48,66,74,67)(52,61,79,62)(55,73)(57,71,84,72)(59,68)(63,80,88,81)(64,78)(69,76,91,77)(70,86)(75,83)(82,100,92,90)(85,96,95,87)(89,103,99,94)(93,107,102,101)(97,105,104,98)(108,109)(110,113)(111,112)(114,115)(117,118,119,120)(123,129)(124,130)(125,133,141,149)(131,143)(132,146,148,162)(136,137)(138,140,154,156)(139,158,155,174)(144,153,159,170)(145,151,161,167)(147,166,163,150)(152,165,169,180)(157,160,173,176)(164,171,178,185)(168,186,194,187)(172,181,199,182)(175,193)(177,191,204,192)(179,188)(183,200,208,201)(184,198)(189,196,211,197)( [...] )

5

In [49]:
permCoxeterWord := function(W, word)
    if word = [] then return (); fi;
    return Product(Data(W).perms{word});
end;

function( W, word ) ... end

In [50]:
permCoxeterWord(E8, word);

(1,121)(2,122)(3,9)(4,10)(5,13,21,29)(6,7,8,142)(11,23)(12,26,28,42)(14,15,134,135)(16,17)(18,20,34,36)(19,38,35,54)(22,126,127,128)(24,33,39,50)(25,31,41,47)(27,46,43,30)(32,45,49,60)(37,40,53,56)(44,51,58,65)(48,66,74,67)(52,61,79,62)(55,73)(57,71,84,72)(59,68)(63,80,88,81)(64,78)(69,76,91,77)(70,86)(75,83)(82,100,92,90)(85,96,95,87)(89,103,99,94)(93,107,102,101)(97,105,104,98)(108,109)(110,113)(111,112)(114,115)(117,118,119,120)(123,129)(124,130)(125,133,141,149)(131,143)(132,146,148,162)(136,137)(138,140,154,156)(139,158,155,174)(144,153,159,170)(145,151,161,167)(147,166,163,150)(152,165,169,180)(157,160,173,176)(164,171,178,185)(168,186,194,187)(172,181,199,182)(175,193)(177,191,204,192)(179,188)(183,200,208,201)(184,198)(189,196,211,197)( [...] )

### Descents

* $s \in S$ is a **left descent** of $w \in W$ if $\ell(sw) < \ell(w)$, i.e., if $\alpha_s.w \in \Phi^{-}$.

In [51]:
isLeftDescent := function(W, w, s)
    return s^w > Data(W).N;
end;

function( W, w, s ) ... end

In [53]:
isLeftDescent(E8, perm, 5);
isLeftDescent(E8, perm, 2);

false

true

### Reduced Expressions

* Conversion `perm -> word`:  $w$ as a word in $S$ is a sequence of left descents.

In [54]:
coxeterWord := function(W, w)
    local word, a;
    word := [];
    while w <> () do
        a := First([1..Data(W).rank], s-> isLeftDescent(W, w, s));
        Add(word, a);  w := Data(W).perms[a] * w;
    od;
    return word;
end;

function( W, w ) ... end

In [55]:
reduced := coxeterWord(E8, perm);

[ 1, 2, 8, 7, 6 ]

In [56]:
permCoxeterWord(E8, reduced) = perm;

true

* To find a reduced expression of any word in $S$, convert `word -> perm` and then `perm -> word`.

In [57]:
reducedWord := function(W, word)
    return coxeterWord(W, permCoxeterWord(W, word));
end;

function( W, word ) ... end

In [59]:
word;
reducedWord(E8, word);

[ 5, 2, 3, 8, 7, 5, 3, 1, 6 ]

[ 1, 2, 8, 7, 6 ]

### Longest Elements

* **Longest elements**.  A finite Coxeter group $W$ has a longest element $w_0$.
* In fact, for each $J \subseteq S$ there is one, $w_J$.
* It can be computed as a product of non-descents. 

In [60]:
longestElement := function(W, J)
    local  wJ,  s;
    wJ := ();
    while true do
        s := First(J, s-> not isLeftDescent(W, wJ, s));
        if s = fail then  return wJ;  fi;
        wJ := W.(s) * wJ;
    od;
end;

function( W, J ) ... end

In [63]:
J:= [4,5,6];
wJ:= longestElement(E8, J);
coxeterWord(E8, wJ);

[ 4, 5, 6 ]

(2,26)(3,27)(4,126)(5,125)(6,124)(7,28)(9,31)(10,18)(11,19)(12,133)(13,132)(14,21)(15,36)(16,24)(17,48)(20,140)(22,29)(23,52)(25,40)(30,45)(41,61)(44,63)(46,66)(49,55)(50,68)(51,57)(53,59)(54,73)(56,62)(58,82)(60,67)(64,76)(65,87)(72,83)(75,89)(80,85)(81,94)(84,96)(86,90)(88,92)(91,106)(95,103)(102,110)(104,111)(105,108)(107,109)(115,118)(116,117)(122,146)(123,147)(127,148)(129,151)(130,138)(131,139)(134,141)(135,156)(136,144)(137,168)(142,149)(143,172)(145,160)(150,165)(161,181)(164,183)(166,186)(169,175)(170,188)(171,177)(173,179)(174,193)(176,182)(178,202)(180,187)(184,196)(185,207)(192,203)(195,209)(200,205)(201,214)(204,216)(206,210)(208,212)(211,226)(215,223)(222,230)(224,231)(225,228)(227,229)(235,238)(236,237)

[ 4, 5, 4, 6, 5, 4 ]

* $w_J$ acts as a graph automorphism on $W_J$.

In [65]:
w6:= longestElement(E8, [1..6]);
Permutation(w6, Data(E8).perms{[1..6]}, OnPoints);

(1,126)(2,122)(3,125)(4,124)(5,123)(6,121)(7,97)(9,133)(10,130)(11,132)(12,131)(13,129)(14,93)(15,101)(16,140)(17,138)(18,137)(19,139)(20,136)(21,89)(22,98)(23,146)(24,147)(25,145)(26,143)(27,144)(28,85)(29,94)(30,153)(31,151)(32,152)(33,150)(34,82)(35,80)(36,90)(37,160)(38,158)(39,75)(40,157)(41,76)(42,87)(43,86)(44,168)(45,165)(46,70)(47,81)(48,164)(49,71)(50,83)(51,172)(52,171)(53,64)(54,77)(55,66)(56,78)(57,177)(60,72)(62,73)(63,183)(69,189)(74,118)(79,117)(84,116)(88,115)(91,114)(92,113)(95,111)(96,112)(99,109)(100,110)(102,107)(103,108)(127,217)(134,213)(135,221)(141,209)(142,218)(148,205)(149,214)(154,202)(155,200)(156,210)(159,195)(161,196)(162,207)(163,206)(166,190)(167,201)(169,191)(170,203)(173,184)(174,197)(175,186)(176,198)(180,192)(182,193)(194,238)(199,237)(204,236)(208,235)( [...] )

(1,6)(3,5)

###  Prefixes

* Prefixes (aka weak Bruhat Order): $u \in W$ is a **prefix** of $w \in W$ if $\ell(u) + \ell(u^{-1} w) = \ell(w)$, i.e., if $w = uv$ with $\ell(w) = \ell(u) = \ell(v)$.
The set of all prefixes of $w$ is the **orbit (!)** of $w$ under the action $(w, s) \mapsto ws$ if $\ell(ws) < \ell(w)$, and $w$ else. (Who is acting? On what?)

In [66]:
prefixes := function(W, w)
    local onRightDescents;

    onRightDescents := function(w, s)
        if isLeftDescent(W, w^-1, s) then
            return w * W.(s);
        else
            return w;
        fi;
    end;

    return orbit([1..Data(W).rank], w, onRightDescents);
end;

function( W, w ) ... end

In [69]:
w := longestElement(E8, [4,5]);;
pre := prefixes(E8, w);;
List(pre, x -> coxeterWord(E8, x));

[ [ 4, 5, 4 ], [ 4, 5 ], [ 5, 4 ], [ 4 ], [ 5 ], [  ] ]

### Parabolic Subgroups and Distinguished Coset Representatives

* For a subset $J \subseteq S$, the subgroup $W_J = \langle J \rangle$ is called a **standard parabolic subgroup** of $W$.
* Each coset $W_J w$, $w \in W$, contains a **unique element of minimal length**.
* The subgroup $W_J$ thus has a **distinguished transversal** $X_J = \{ x \in W : \ell(ux) = \ell(u) + \ell(x) \text{ for all } u \in W_J \}$.
* In fact, $X_J$ is the set of prefixes of the **longest coset representative** $d_J = w_J^{-1} w_S = w_J w_S$.

In [70]:
longestCosetElement := function(W, J, L)
    return longestElement(W, J) * longestElement(W, L);
end;

function( W, J, L ) ... end

* For example, the longest coset representative of $E_6 \subset E_7$ (in $E_8$).

In [72]:
dJL := longestCosetElement(E8, [1..6], [1..7]);;
word := coxeterWord(E8, dJL);

[ 7, 6, 5, 4, 2, 3, 1, 4, 3, 5, 4, 2, 6, 5, 4, 3, 1, 7, 6, 5, 4, 2, 3, 4, 5,   6, 7 ]

* And here is a function that computes the distinguished transversal of a parabolic subgroup $W_J$ of $W$:

In [73]:
parabolicTransversal := function(W, J)
    return prefixes(W, longestCosetElement(J, [1..Data(W).rank]));
end;

function( W, J ) ... end

* Naturally, $X_J$ is a graph.

In [74]:
prefixes_with_edges := function(W, w)
    local onRightDescents;

    onRightDescents := function(w, s)
        if isLeftDescent(W, w^-1, s) then
            return w * W.(s);
        else
            return w;
        fi;
    end;

    return orbit_with_edges([1..Data(W).rank], w, onRightDescents);
end;

function( W, w ) ... end

In [77]:
XJ := prefixes_with_edges(E8, dJL);;
edges := Filtered(XJ.edges, x-> x[1] <> x[2]);;
PlotGraph(edges, opts);

##  Shapes: Classes of Parabolic Subgroups

* $W$ acts on its subgroups by conjugations, in particular on its parabolic subgroups.
* The set of standard parabolic subgroups $W_J$, $J \subseteq S$ is thus partitioned into **shapes**: classes of conjugate parabolics.

###  Longest Words Action

* Obviously, if $L = J \cup \{s\}$ for some $s \in S \setminus J$ then conjugation by $d_J^L$ maps $W_J$ to
$W_K$ for some $K \subseteq L$.  In fact, $K = J^{d_J^L}$.
* Define an action of (the free monoid) $S^*$ on the power set $2^S$ as
$$
(J, s) \mapsto J.s := J^{d_J^L}, \quad L = J \cup \{s\}.
$$
* There is an **Orbit-Stabilizer Theorem** for this action.

<div class="alert alert-danger">

**Theorem** (Howlett, ...) Let $(W, S)$ be a finite Coxeter group, and let $S^*$ act on $2^S$ as above. Then:
* The shape $[J]$ of $J \subseteq S$ is the $S^*$-orbit of $J$ under the above action.
* $N_W(W_J) = W_J \rtimes N_J$, where $N_J = \{x \in X_J : J^x = J\}$ is the stablizer in $W$ of $J$ under this action.
    
</div>

* Let's see this in action, shapes first.
* Actually, the tack-on action $(J, s) \mapsto J \cup \{s\}$ first.

In [78]:
tackOn := function(x, s)
    return Union(x, [s]);
end;

function( x, s ) ... end

In [79]:
tackOn([1,3,5], 4);

[ 1, 3, 4, 5 ]

* Using the longest words action on parabolics, we can make the shape of $J \subseteq$ as an $S^*$-orbit.
* Here, the action function `onParabolics` is local to the function `shape`, since it needs access to the ambient group $W$.

In [80]:
shape := function(W, J)
    local onParabolics;
    onParabolics := function(K, s)
        return OnSets(K, longestCosetElement(W, K, tackOn(K, s)));
    end;
    return orbit([1..Data(W).rank], J, onParabolics);
end;

function( W, J ) ... end

In [81]:
shape(E8, [1,3]);

[ [ 1, 3 ], [ 3, 4 ], [ 2, 4 ], [ 4, 5 ], [ 5, 6 ], [ 6, 7 ], [ 7, 8 ] ]

 * Denote by $\Lambda$ the set of all shapes of $W$.
 * Recall (from the exercise in part 1) how the power set $2^S$ is the orbit of $S^*$ under the `takeAway` action.
 * Now, $\Lambda$ is the $S^*$-orbit of the shape $\{S\}$ under that same action.

In [82]:
takeAway := function(x, s)
    return Difference(x, [s]);
end;

function( x, s ) ... end

In [83]:
shapes := function(W)
    local onShapes, S;
    onShapes := function(x, s)
        return Set(shape(W, takeAway(x[1], s)));
    end;
    S := [1..Data(W).rank];
    return orbit(S, shape(W, S), onShapes);
end;

function( W ) ... end

* $E_8$, for example:

In [87]:
sss := shapes(E8);;
List(sss, x -> x[1]);
Length(sss);
Sum(sss, Length);

[ [ 1 .. 8 ], [ 2, 3, 4, 5, 6, 7, 8 ], [ 1, 3, 4, 5, 6, 7, 8 ],   [ 1, 2, 4, 5, 6, 7, 8 ], [ 1, 2, 3, 5, 6, 7, 8 ], [ 1, 2, 3, 4, 6, 7, 8 ],   [ 1, 2, 3, 4, 5, 7, 8 ], [ 1, 2, 3, 4, 5, 6, 8 ], [ 1, 2, 3, 4, 5, 6, 7 ],   [ 1, 3, 4, 5, 6, 7 ], [ 1, 2, 3, 4, 6, 8 ], [ 1, 3, 4, 6, 7, 8 ],   [ 2, 3, 4, 5, 7, 8 ], [ 1, 2, 3, 4, 5, 7 ], [ 2, 3, 4, 5, 6, 7 ],   [ 1, 2, 4, 5, 6, 7 ], [ 1, 2, 3, 4, 6, 7 ], [ 1, 2, 3, 5, 6, 7 ],   [ 1, 2, 3, 5, 6, 8 ], [ 1, 2, 3, 4, 5, 6 ], [ 1, 3, 4, 5, 6 ],   [ 1, 2, 3, 4, 6 ], [ 1, 3, 4, 6, 7 ], [ 1, 2, 4, 5, 7 ], [ 1, 2, 3, 5, 7 ],   [ 2, 3, 4, 5, 7 ], [ 1, 2, 3, 4, 5 ], [ 1, 2, 3, 5, 6 ], [ 1, 2, 3, 4 ],   [ 1, 2, 4, 5 ], [ 1, 3, 5, 6 ], [ 1, 2, 3, 5 ], [ 1, 2, 5, 7 ],   [ 2, 3, 4, 5 ], [ 1, 3, 4 ], [ 1, 2, 3 ], [ 1, 2, 5 ], [ 1, 3 ], [ 1, 2 ],   [ 1 ], [  ] ]

41

256

###  Shape Graph

* Naturally, a shape has a graph.

In [88]:
shape_with_edges := function(W, J)
    local onParabolics;
    onParabolics := function(K, s)
        return OnSets(K, longestCosetElement(W, K, tackOn(K, s)));
    end;
    return orbit_with_edges([1..Data(W).rank], J, onParabolics);
end;

function( W, J ) ... end

In [91]:
sh := shape_with_edges(E8, [1,3]);;
edges := Filtered(sh.edges, x-> x[1] <> x[2]);;
PlotGraph(edges);

* In order to compute a **transversal** for the shape, we need to modify the algorithm.
* We need to adjust for the fact that a generator $s \in S \subseteq \mathbb{N}$ merely represents a conjugating element $d_K^L \in W$, which depends on the context.
* We might then as well also take into account, that for the current parabolic $K$ only $s \notin K$ needs to be considered as generator.

In [92]:
shape_with_transversal := function(W, J)
    local   S,  list,  reps,  i,  K,  s,  a,  L;
    S := [1..Data(W).rank];  list := [J];  reps := [()];  i := 0;
    while i < Length(list) do
        i := i+1;  K := list[i];
        for s in Difference(S, K) do
            a := longestCosetElement(W, K, tackOn(K, s));
            L := OnSets(K, a);
            if not L in list then
                Add(list, L);
                Add(reps, reps[i] * a);
            fi;
        od;
    od;
    return rec(list := list, reps := reps);
end;

function( W, J ) ... end

In [93]:
List(shape_with_transversal(E8, [1,3]).reps, x -> coxeterWord(E8, x));

[ [  ], [ 4, 3, 1 ], [ 4, 2, 3, 1, 4, 3 ], [ 4, 3, 1, 5, 4, 3 ],   [ 4, 3, 1, 5, 4, 3, 6, 5, 4 ], [ 4, 3, 1, 5, 4, 3, 6, 5, 4, 7, 6, 5 ],   [ 4, 3, 1, 5, 4, 3, 6, 5, 4, 7, 6, 5, 8, 7, 6 ] ]

##  Normalizers of Parabolic Subgroups

* Recall that $N_W(W_J) = W_J \rtimes N_J$, and let us compute the complement $N_J$ as the **stabilizer** of $J$ in the action of $S^*$ on $2^S$
* For this, we compute a transversal as before, and additionally Schreier generators, one for each non-tree edge in the action graph.
* We distinguish two kinds of Schreier generators: **ears**, if the edge is a loop arising from $K.s = K$, and **eyes**, if the edge closes a proper cycle with $K.s \neq K$.
* Howlett has shown that the **ears** naturally generate a Coxeter group.  And that nontrivial **eyes** are rare.

In [94]:
parabolicComplement := function(W, J)
    local   S,  list,  reps,  i,  gens,  K,  s,  a,  L,  j;
    S := [1..Data(W).rank];  list := [J];  reps := [()];
    gens := rec(ears := [], eyes := []);
    i := 0;
    while i < Length(list) do
        i := i+1;  K := list[i];
        for s in Difference(S, K) do
            a := longestCosetElement(W, K, tackOn(K, s));
            L := OnSets(K, a);  j := Position(list, L);
            if j = fail then
                Add(list, L);
                Add(reps, reps[i] * a);
            elif j = i then
                AddSet(gens.ears, reps[i] * a * reps[i]^-1);
            else
                AddSet(gens.eyes, reps[i] * a * reps[j]^-1);
            fi;
        od;
    od;
    return gens;
end;

function( W, J ) ... end

In [95]:
parabolicComplement(E8, [1,3]);

rec( ears := [ (7,15)(8,128)(14,22)(21,29)(28,36)(34,42)(35,43)(39,47)(41,50)(46,54)(49,56)(53,60)(55,62)(58,65)(59,67)(61,68)(64,72)(66,73)(70,77)(71,78)(75,81)(76,83)(80,86)(82,87)(85,90)(89,94)(93,98)(97,101)(119,120)(127,135)(134,142)(141,149)(148,156)(154,162)(155,163)(159,167)(161,170)(166,174)(169,176)(173,180)(175,182)(178,185)(179,187)(181,188)(184,192)(186,193)(190,197)(191,198)(195,201)(196,203)(200,206)(202,207)(205,210)(209,214)(213,218)(217,221)(239,240), (6,14)(7,127)(8,15)(13,21)(20,28)(26,34)(27,35)(31,39)(33,41)(38,46)(40,49)(45,53)(48,55)(51,58)(52,59)(57,64)(63,70)(68,74)(69,75)(73,79)(78,84)(83,88)(86,91)(87,92)(90,95)(94,99)(98,102)(101,104)(118,119)(126,134)(128,135)(133,141)(140,148)(146,154)(147,155)(151,159)(153,161)(158,166)(160,169)(165,173)(168,175)(171,178)(172,179)(177,184)(183,190)(188,194)(189,195)(193,199)(198,204)(203,208)(206,211)(207,212)(210,215)(214,219)(218,222)(221,224)(238,239), (5,13)(6,126)(7,14)(12,20)(15,22)(18,26)(19,27)(24,31)(25,33)(30,3

##  Conjugacy Class by Length

### Minimal Length Representatives

* For a conjugacy class $C$ of $W$ denote by $C_{\mathrm{min}}$ the set of elements of **minimal length** in $C$.
* For $w \in W$ and $s \in S$ denote $w \longrightarrow w^s$ if $\ell(w^s) \leq \ell(w)$ and extend notation to the reflexive and transitive closure.
* A conjugacy class $C$ is called **cuspidal** if $C \cap W_J = \emptyset$ for all $J \subsetneq S$.
* For $w \in W$, denote by $J(w)$ the set of simple reflections $s \in S$ occuring in a reduced expression for $w$.
* The following property of $C_{\mathrm{min}}$ forms the basis of the definition of a square **character table** for the Iwahori-Hecke algebra of $W$ ...

<div class="alert alert-danger">

**Theorem** (Geck-Pfeiffer)**.**  Let $(W, S)$ be a finite Coxeter group.
* $C \longrightarrow C_{\mathrm{min}}$ for each conjugacy class $C$ of $W$: for each $w \in C$ there is a $v \in C_{\mathrm{min}}$ such that $w \longrightarrow v$.
* The set $\{J(w) : w \in C_{\mathrm{min}}\}$ forms a shape of $W$.
* The conjugacy classes of $W$ are parametrized by pairs $([J], D)$ where $[J]$ is shape and $D$ is a cuspidal class of $W_J$.
</div>

* To find the minimal length conjugates of $x \in W$, we modify the orbit algorithm so that it **forgets the current orbit** whenever a shorter conjugate is found.

In [96]:
minConjugates := function(W, x)
    local   list,  lx,  i,  S,  y,  s,  z,  lz;
    list := [x];  lx := coxeterLength(W, x);
    i := 0;  S := [1..Data(W).rank];
    while i < Length(list) do
        i := i+1;  y := list[i];
        for s in S do
            z := y^W.(s);  lz := coxeterLength(W, z);
            if lz = lx then
                if not z in list then  Add(list, z);  fi;
            elif lz < lx then
                list := [z];  lx := lz;  i := 0;  break;
            fi;
        od;
    od;
    return list;
end;

function( W, x ) ... end

* test it on a random element

In [99]:
W := coxeterGroup(cartanMat("E", 7));
w := randomGroupElement(W);;
coxeterWord(W, w);

<permutation group with 7 generators>

[ 3, 4, 3, 5, 4, 2, 6, 5, 4, 2, 3, 1, 4, 7, 6, 5, 4, 2, 3, 1, 4, 3, 5, 6 ]

In [101]:
min := minConjugates(W, w);;
List(min, x -> coxeterLength(W, x));

[ 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,   6, 6, 6, 6, 6, 6, 6 ]

### Conjugacy Classes

* We can use this and the fact that the **supports** of $x \in C_{\mathrm{min}}$ form a shape $\Lambda$ to compute a **canonical representative** of minimal length for each conjugacy class $C$ of $W$, starting from any element $w \in C$, as follows.

In [102]:
coxeterMinRep := function(W, w)
    local   v,  K,  sh,  j;
    v := minConjugates(W, w)[1];
    K := Set(coxeterWord(W, v));
    sh := shape_with_transversal(W, K);
    j := PositionMinimum(sh.list);
    return Minimum(minConjugates(W, v^sh.reps[j]));
end;

function( W, w ) ... end

In [103]:
coxeterWord(W, coxeterMinRep(W, w));

[ 7, 6, 5, 4, 2, 3 ]

*  Acting on the canonical representatives with a generating set that is closed under conjugation, we enumerate **all the conjugacy classes** of $W$.

In [104]:
coxeterConjugacyClasses := function(W)
    local   onMinReps;
    onMinReps := function(x, a)
        return coxeterMinRep(W, x * a);
    end;
    return orbit(reflections(W), (), onMinReps);
end;

function( W ) ... end

In [106]:
cc := coxeterConjugacyClasses(W);;
List(cc, x -> coxeterLength(W, x));

[ 0, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 12, 6, 5, 5, 7, 5, 5, 5, 5,   13, 5, 5, 5, 7, 13, 6, 6, 8, 6, 8, 6, 8, 14, 6, 6, 12, 10, 6, 24, 14, 14,   16, 30, 7, 9, 11, 15, 17, 13, 25, 23, 21, 33, 31, 63 ]

* Finding the conjugacy classes of $E_8$ takes about 30 minutes ...

## Exercises, etc.

* Check that the function `permutation`, when it is used to map matrices to their effect on the root system, is a homomorphism.

* ($*$) Find an efficient way to enumerate **double cosets** $X_{JK} = X_J \cap X_K^{-1}$, $J, K \subseteq S$.

* Prove that the shapes set $\Lambda$ is an orbit under `takeAway`.

* ($*$) Rewrite `shape_with_edges` as `shape_with_images`, collecting the labelled, directed edge information as image lists.  What are the resulting permutations?

* Prove that the complement $N_J$ is a stabiliser under `onParabolics`.

* Find an example of a parabolic subgroup $W_J$ whose normalizer complement $N_J$ contains a set of eyes that generates a subgroup of size more than $2$.  Is that group elementary abelian?

* **Cyclic Shift Classes.** We say that $w, v \in W$ are **conjugate by cyclic shift** and write $w \leftrightarrow v$ if both $w \longrightarrow v$ and $v \longrightarrow w$.  Show that this relation is the equivalence relation generated by: $ws \leftrightarrow sw$ if $\ell(ws) = \ell(sw)$, for $w \in W$, $s \in S$.  Write a version of the orbit algorithm that computes the cyclic shift class of a given $w \in W$, i.e., the set $\{v \in W : w \leftrightarrow v\}$, together with the edges $vs \leftrightarrow sv$.

In [109]:
Read("coxeter.g");
Read("shifts.g");
E6 := coxeterGroup(cartanMat("E", 6));

<permutation group with 6 generators>

In [113]:
w := randomGroupElement(E6);;
cyc := cyclic_shifts_with_edges(E6, w);;
edges := Filtered(Set(cyc.edges), x -> x[1] <> x[2]);;
PlotGraph(edges);

* ($*$) Provide a version of the orbit algorithm that, for a given element $w \in W$ computes the partition of its conjugacy class in $W$ into cyclic shift classes, together with with graph induced by the relation $\longleftarrow$ on the quotient set.

* ($*$) Define an equivalence relation $\approx$ on $W$ as appropriate closure of: $uv \approx vu$ if $\ell(uv) = \ell(u) + \ell(v) = \ell(v) + \ell(u) = \ell(vu)$, $u,v \in W$.  Show that a $\approx$-class is a union of cyclic shift classes.  Find an example of a $\approx$-class that is not a (single) cyclic shift class.

* **Involutions.**  An element $w \in W$ is an **involution** if $w^2 = 1$.  Show that if $w \in W$ is an involution then its cyclic shift class is just $\{w\}$, i.e., that for $s \in S$, either $\ell(sws) <> \ell(w)$ or $sws = w$.

* Let $I(W) = \{w \in W : w^2 = 1\}$ be the set of all involutions in $W$. Show that the monoid $S^*$ acts on $I(W)$ via
$$
w.s = \begin{cases}
ws & \text{if } \ell(sws) = \ell(w),\\
sws & \text{else.}
\end{cases}
$$
Show that $I(W)$ is the orbit of the indentity element $1$ of $W$ under this action.  Use a suitable version of the orbit algorithm to compute $I(W)$ as orbit of $1$.

In [118]:
Read("coxeter.g");
Read("involution.g");
A3 := coxeterGroup(cartanMat("A", 3));;
invo := involutions(A3);
List(invo, x -> coxeterWord(A3, x));

[ (), (1,7)(2,4)(5,6)(8,10)(11,12), (1,4)(2,8)(3,5)(7,10)(9,11),   (2,5)(3,9)(4,6)(8,11)(10,12), (1,8)(2,7)(3,6)(4,10)(9,12),   (1,7)(2,6)(3,9)(4,5)(8,12)(10,11), (1,6)(2,9)(3,8)(5,11)(7,12),   (1,11)(3,10)(4,9)(5,7)(6,12), (1,3)(2,12)(4,10)(5,11)(6,8)(7,9),   (1,9)(2,8)(3,7)(4,11)(5,10)(6,12) ]

[ [  ], [ 1 ], [ 2 ], [ 3 ], [ 1, 2, 1 ], [ 1, 3 ], [ 2, 3, 2 ],   [ 1, 2, 3, 2, 1 ], [ 2, 1, 3, 2 ], [ 1, 2, 1, 3, 2, 1 ] ]

* Formulate an action (of $S^*$) on the conjugacy classes of involutions of $W$ that can be used to determine all conjugacy classes of involutions of $W$ as orbit of the conjugacy class of the trivial element.  Verify in examples that the conjugacy classes of involutions correspond to the shapes of those parabolic subgroups $W_K$ whose longest element $w_K$ is central in $W_K$.

In [122]:
Read("coxeter.g");
Read("involution.g");
W := coxeterGroup(cartanMat("A", 3));;
involutionClasses(W);

[ ()^G, (1,7)(2,4)(5,6)(8,10)(11,12)^G, (1,7)(2,6)(3,9)(4,5)(8,12)(10,11)^G ]

In [123]:
Filtered(shapes(W), sh -> longestElement(W, sh[1]) in Centre(Subgroup(W, GeneratorsOfGroup(W){sh[1]})));

[ [ [ 1, 3 ] ], [ [ 1 ], [ 2 ], [ 3 ] ], [ [  ] ] ]